In [1]:
# Imports
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# Reading the training data
data_train = pd.read_csv('ex3_train.csv', sep=",", encoding='utf-8', header='infer')
df_train = data_train

y_train = df_train['y']
X_train = df_train.drop(['y'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(y_train.value_counts())

(3500, 400)
(3500,)
2    361
8    357
3    353
9    351
7    350
5    349
1    349
0    345
6    343
4    342
Name: y, dtype: int64


In [3]:
# Reading the test data
data_test = pd.read_csv('ex3_test.csv', sep=",", encoding='utf-8', header='infer')
df_test = data_test

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)
print(X_test.shape)
print(y_test.shape)
print(y_test.value_counts())

(1500, 400)
(1500,)
4    158
6    157
0    155
5    151
1    151
7    150
9    149
3    147
8    143
2    139
Name: y, dtype: int64


In [4]:
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print(classification_report(y_test, y_pred))

Accuracy of logistic regression classifier on test set: 0.91
[[151   0   1   0   1   0   1   0   1   0]
 [  0 143   0   1   1   3   0   0   3   0]
 [  1   1 124   2   0   3   2   1   4   1]
 [  0   0  10 128   0   5   0   3   0   1]
 [  1   2   2   1 143   0   3   0   2   4]
 [  4   0   0  10   2 125   0   0   9   1]
 [  0   0   1   0   0   4 152   0   0   0]
 [  0   1   2   0   3   0   0 137   0   7]
 [  0   1   3   3   1   4   1   0 129   1]
 [  1   1   0   1   4   1   2   9   2 128]]
             precision    recall  f1-score   support

          0       0.96      0.97      0.96       155
          1       0.96      0.95      0.95       151
          2       0.87      0.89      0.88       139
          3       0.88      0.87      0.87       147
          4       0.92      0.91      0.91       158
          5       0.86      0.83      0.84       151
          6       0.94      0.97      0.96       157
          7       0.91      0.91      0.91       150
          8       0.86      0.